
![alt text](https://www.business.unsw.edu.au/style%20library/asb/assets/images/logo-unsw.png)
__________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [52]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
#from google.colab import files

import matplotlib.pyplot as plt
#import seaborn as sns; sns.set()
import datetime
%matplotlib inline 

In [53]:
from p_values import Variance_Inflation_Factor
def variance_inflation_factor(x):
    columns = x.columns
    VIF = Variance_Inflation_Factor(x)
    df_vif = pd.DataFrame({"vif": VIF})
    df_vif = df_vif.set_index(columns)
    return df_vif

In [54]:
import calendar
import re
import pandas as pd, numpy as np
from datetime import date

In [55]:
remove_pict = [ x[4:] for x in ['avg_Depth of field_hue_Picture',
 'avg_Depth of field_saturation_Picture',
 'avg_Depth of field_value_Picture',"avg_Color_balance_horizontal_Picture", "avg_Color_balance_vertical_Picture",
               "avg_Clarity_Picture","avg_Size_difference_Picture",
               "avg_Saturation_Picture","avg_Color_difference_Picture",
                "avg_Texture_difference_Picture","avg_ROT_Picture","avg_Physical_ver_Picture"]]

In [56]:
#getting len text 
data_len = pd.read_csv("dhckorea2526posts_text_related_var_final.csv", encoding='ISO-8859–1' )

In [57]:
#getting nre data from yonsei
data_yonsei = pd.read_excel("DHC Insta1426_20200823.xlsx")
data_yonsei.columns = data_yonsei.iloc[0]
data_yonsei = data_yonsei.iloc[1:]

In [58]:
data_yonsei.shape

(1426, 141)

In [59]:
subset_yonsei = ["shortcode","numPos","numNeg",
"numFocalPd",
"numTotPd",
"numBrand",
"numModel",
"numOYBrand"
]


In [60]:
share_num = data_yonsei.loc[:,subset_yonsei]
share_num["short_code"] = share_num["shortcode"].apply(lambda x:x+".jpg")
share_num = share_num.set_index('short_code')
share_num["numPos-Neg"] = share_num["numPos"]-share_num["numNeg"]
#share_num["num_PdMention"] = share_num["numFocalPd"]+share_num["numTotPd"]
share_num = share_num.drop(columns=["numTotPd","shortcode","numPos","numNeg"])

In [61]:
share_num.head()

,numFocalPd,numBrand,numModel,numOYBrand,numPos-Neg
short_code,,,,,
BXB7JudD_dF.jpg,0,4,0,0,1
BCbtbYNtLkI.jpg,4,1,0,0,1
BCePOq-NLrG.jpg,0,7,0,0,2
BCgy0CeNLkf.jpg,0,4,0,0,2
BCkZZ27tflk.jpg,0,2,0,0,4


In [62]:
aug2020_cv = pd.read_excel("aug2020_control_variables.xlsx").set_index("short_code")

In [63]:
Data_a = pd.read_excel("20201104_tv-SMData-dummydayshourweek.xlsx")
Data_b = Data_a.loc[~Data_a["short_code"].isin(["BcCy8B1lzCc.jpg","BcGN3tYh4Rj.jpg","BcENYeOgVs2.jpg"])]
Data = Data_a.set_index("short_code")


In [64]:
Data_a.shape

(1426, 154)

In [65]:
column_to_drop = remove_pict

In [66]:
Data.shape

(1426, 153)

In [67]:
all_pict =  [ x for x in Data.columns if re.search("Picture",x)]

In [68]:
all_pict

['Colorfulness_Picture',
 'Saturation_Picture',
 'Contrast_Picture',
 'Clarity_Picture',
 'Brightness_Picture',
 'Warm Hue_Picture',
 'ROT_Picture',
 'Diagonal_dominance_Picture',
 'Physical_ver_Picture',
 'Physical_Hor_Picture',
 'Color_balance_vertical_Picture',
 'Color_balance_horizontal_Picture',
 'Texture_difference_Picture',
 'Size_difference_Picture',
 'Color_difference_Picture',
 'Depth of field_hue_Picture',
 'Depth of field_saturation_Picture',
 'Depth of field_value_Picture']

In [69]:
#Data11= Data.drop(column_to_drop, axis=1)
Data11=Data

In [70]:
Data2 = Data11.join(aug2020_cv)

In [71]:
Data2.columns

Index(['Week', 'model_exist', 'taken_at_timestamp', 'Colorfulness_Picture',
       'Saturation_Picture', 'Contrast_Picture', 'Clarity_Picture',
       'Brightness_Picture', 'Warm Hue_Picture', 'ROT_Picture',
       ...
       'dummy_Week_94', 'dummy_Week_95', 'dummy_Week_96', 'log_comment',
       'log_followers', 'posts', 'smile', 'post_age', 'image_size_kilobytes',
       'tag_numb'],
      dtype='object', length=160)

In [72]:
share_num.columns

Index(['numFocalPd', 'numBrand', 'numModel', 'numOYBrand', 'numPos-Neg'], dtype='object', name=0)

In [73]:
share_num2 = share_num.join(Data2["log_len_text_total"])
share_num2["len_text_total"] = share_num2["log_len_text_total"].apply(lambda x:np.exp(x))
for col in share_num.columns:
    if col in ['numPos', 'numNeg']:
        #pass
        #share_num2[col] = share_num2[col].apply(lambda x:1 if x>1 else 0)
        share_num2[col] = share_num2[col]/share_num2["len_text_total"]
    else:
        #pass
        #share_num2[col] = share_num2[col].apply(lambda x:1 if x>1 else 0)
        share_num2[col] = share_num2[col]/share_num2["len_text_total"]
share_num2 = share_num2.drop(columns=["log_len_text_total","len_text_total"])

# binary numShare

drop_ori=list()
for col in share_num2:
    if col=="len_text_total":
        continue
    else:
        drop_ori.append(col)
        share_num2[col+"_binary"]=share_num2[col].apply(lambda x:1 if x>0  else 0)
        #share_num2[col+"_binary/txt"] = share_num2[col+"_binary"] / share_num2["len_text_total"]
        #drop_ori.append(col+"_binary")
share_num2 = share_num2.drop(columns=drop_ori+["len_text_total"])

In [74]:
share_num2.head()

,numFocalPd,numBrand,numModel,numOYBrand,numPos-Neg
short_code,,,,,
BXB7JudD_dF.jpg,0,0.0175439,0,0,0.00438596
BCbtbYNtLkI.jpg,0.0148699,0.00371747,0,0,0.00371747
BCePOq-NLrG.jpg,0,0.030837,0,0,0.00881057
BCgy0CeNLkf.jpg,0,0.0188679,0,0,0.00943396
BCkZZ27tflk.jpg,0,0.0123457,0,0,0.0246914


In [75]:
aug2020_cv.columns

Index(['log_comment', 'log_followers', 'posts', 'smile', 'post_age',
       'image_size_kilobytes', 'tag_numb'],
      dtype='object')

In [76]:
[x for x in Data2.columns if re.search("Picture",x)]

['Colorfulness_Picture',
 'Saturation_Picture',
 'Contrast_Picture',
 'Clarity_Picture',
 'Brightness_Picture',
 'Warm Hue_Picture',
 'ROT_Picture',
 'Diagonal_dominance_Picture',
 'Physical_ver_Picture',
 'Physical_Hor_Picture',
 'Color_balance_vertical_Picture',
 'Color_balance_horizontal_Picture',
 'Texture_difference_Picture',
 'Size_difference_Picture',
 'Color_difference_Picture',
 'Depth of field_hue_Picture',
 'Depth of field_saturation_Picture',
 'Depth of field_value_Picture']

In [77]:
#remove log so you can use non log of following and followers
#for col in ["posts","tag_numb"]:
    #print(Data2[col].min())
    #Data2["log_"+col] = np.log(Data2[col])

In [78]:
#selected_pict = [x for x in Data2.columns if re.search("Picture",x)]

In [79]:
for col in ["Contrast_Picture"]:
    #pass
    avg = Data2[col].mean()
    Data2[col+"_z"] = Data2[col]-avg
    Data2[col+"^2"] = np.square(Data2[col+"_z"]) 

In [80]:
log_IV = [x for x in Data2.columns if re.search("log",x) and not(re.search("_likes",x))]

In [81]:
#remove Log
for col in log_IV:
    Data2[col[4:]] = np.exp(Data2[col])

Data_count = Data2.copy()
Data_count["none_UGC"] =  np.where((Data2["temp"]==0) & (Data2["dummy_motivation_offic"]==0) & (Data2["faces"]==0), 1,0)
Data_count["source"] = np.where(Data_count["dummy_motivation_offic"]==1,"official","user")
Data_count.groupby("source").sum()[["product_only","product_model","model_only","product_users","users_only","none_FGC","none_UGC"]].T

Data2["redudancy"] = Data2.loc[:,["product_only","product_model","model_only","product_users","users_only","none_FGC"]].sum(axis=1)
Data2.loc[Data2["redudancy"]>1]

In [82]:
Data2["product_only"].value_counts()

0    818
1    608
Name: product_only, dtype: int64

In [83]:
#divide 1000 to increase coefficient
for col in ["followers","following","posts"]:
    Data2[col] = Data2[col].apply(lambda x: x/1000)

In [84]:
Data3 = Data2.join(share_num2)

In [85]:
Data3.loc[:,['product_model',
       'product_only', 'model_only',]].sum()

product_model     93
product_only     608
model_only       371
dtype: int64

In [86]:
#Data3.to_excel("Complete_SMdata20200911.xlsx")

In [87]:
share_num.columns

Index(['numFocalPd', 'numBrand', 'numModel', 'numOYBrand', 'numPos-Neg'], dtype='object', name=0)

In [88]:
[x for x in Data3.columns if re.search("motivation",x)]

['dummy_motivation_event', 'dummy_motivation_offic']

In [89]:
Data6 = Data3.drop(["model_exist","product_type","Contrast_Picture","Contrast_Picture^2",
                "image_size_kilobytes","tag_numb",'comment',
                "Week","taken_at_timestamp",'log_comment',
                'post_age',"smile"]+log_IV+remove_pict, axis=1).astype(float)


In [90]:
#for SM Regression
Data6.to_excel("20201123-tv_smvariables.xlsx")

In [91]:
Dstop

NameError: name 'Dstop' is not defined

# for sales

In [ ]:
DataSales = Data3.drop(["model_exist","product_type","Contrast_Picture","Contrast_Picture^2",
                "image_size_kilobytes","tag_numb",'comment',
                'log_comment',
                'post_age',"smile"]+log_IV+remove_pict, axis=1).astype(float)


In [ ]:
DataSales.to_excel("20201104-tv_smvariables-sales.xlsx")